<a href="https://colab.research.google.com/github/haaraamk/DLforAI_class01/blob/main/EfficientNetB4_haram_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# 데이터 path 설정 각자 자기 주소에 맞게 설정하면되긴하는데 그냥 이 주소로 통일 하자
dpath = "/content/drive/MyDrive/CAU_BURGER/data/"

# Introduction to image classification using camera trap images

In [2]:
!pip install efficientnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.5 MB/s eta 0:00:00


In [3]:
from efficientnet.tfkeras import EfficientNetB4, preprocess_input

Next, let's import some of the usual suspects:

In [4]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from tqdm import tqdm

Read in the train and test CSVs first and see what they look like.

In [7]:
train_features = pd.read_csv(dpath+"train_features.csv", index_col="id")
test_features = pd.read_csv(dpath+"test_features.csv", index_col="id")
train_labels = pd.read_csv(dpath+"train_labels.csv", index_col="id")

## 3. Explore the data

## 4. Split into train and evaluation sets
First, we'll need to split the images into train and eval sets. We'll put aside 25% of the data for evaluation and stratify by the target labels to ensure we have similar relative frequencies of each class in the train and eval sets.

In [8]:
from sklearn.model_selection import train_test_split

frac = 0.5

y = train_labels.sample(frac=frac, random_state=1)
x = train_features.loc[y.index].filepath.to_frame()

# note that we are casting the species labels to an indicator/dummy matrix
x_train, x_eval, y_train, y_eval = train_test_split(
    x, y, stratify=y, test_size=0.25
)

In [8]:
# sampling 안하고 모든 데이터 사용
# 이 코드는 나중에 모델링 다 끝났을 때 사용하자. 왜? 처음부터 모든 데이터를 사용하면 훈련하는데 시간이 오래걸리니까

# from sklearn.model_selection import train_test_split

# y = train_labels
# x = train_features.loc[:, 'filepath'].to_frame()

# x_train, x_eval, y_train, y_eval = train_test_split(
#     x, y, stratify=y, test_size=0.25
# )

## 5. Build the Model

모델 : efficientnetb4
가중치 : 동결
fine tuning : x
pre-trained model 후 층 dese층 개수 : 1(256)(활성화함수 : relu)
드롭아웃 : 0.5
optimizer = Adam(learning_rate=0.0001)

몇 번째 epoch에서 멈추었는가 : Epoch 16/100

loss: 0.5626
accuracy: 0.8110
val_loss: 0.6739
val_accuracy: 0.7866

저장한 model 파일 이름 :  EfficientNetB4_haram_1.hd5

제출 후 test loss : 1.6580

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import SGD


In [10]:
class ImagesDataset_train(tf.keras.utils.Sequence):
    """Reads in an image, transforms pixel values, and serves
    a dictionary containing the image id, image tensors, and label.
    """

    def __init__(self, x_df, y_df=None, batch_size=16): # batch_size=16
        self.x = x_df
        self.y = y_df
        self.batch_size = batch_size
        self.indexes = self.x.index.values
        self.num_samples = len(self.indexes)
        self.transform = tf.keras.preprocessing.image.ImageDataGenerator( # 이 부분이 data augmentation
            rescale=1./255,
          rotation_range=10, #
          width_shift_range=0.1,#
          height_shift_range=0.1,#
          shear_range=0.1,#
          zoom_range=0.1,#
          vertical_flip=True,#
          fill_mode='nearest',#
            samplewise_center=True,
            samplewise_std_normalization=True
        )

    def __len__(self):
        return int(self.num_samples / self.batch_size)

    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx*self.batch_size: (idx+1)*self.batch_size]
        batch_x = self.x.loc[batch_indexes].filepath.values
        batch_images = []
        for filepath in batch_x:
            img = image.load_img(dpath+filepath, target_size=(380, 380))
            img = image.img_to_array(img)
            batch_images.append(img)
        batch_images = self.transform.standardize(np.array(batch_images))
        if self.y is not None:
            batch_y = self.y.loc[batch_indexes].values
            return batch_images, batch_y
        else:
            return batch_images


In [11]:
class ImagesDataset(tf.keras.utils.Sequence):
    """Reads in an image, transforms pixel values, and serves
    a dictionary containing the image id, image tensors, and label.
    """

    def __init__(self, x_df, y_df=None, batch_size=16): # batch_size=16
        self.x = x_df
        self.y = y_df
        self.batch_size = batch_size
        self.indexes = self.x.index.values
        self.num_samples = len(self.indexes)
        self.transform = tf.keras.preprocessing.image.ImageDataGenerator( 
            rescale=1./255,
            samplewise_center=True,
            samplewise_std_normalization=True
        )

    def __len__(self):
        return int(self.num_samples / self.batch_size)

    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx*self.batch_size: (idx+1)*self.batch_size]
        batch_x = self.x.loc[batch_indexes].filepath.values
        batch_images = []
        for filepath in batch_x:
            img = image.load_img(dpath+filepath, target_size=(380, 380))
            img = image.img_to_array(img)
            batch_images.append(img)
        batch_images = self.transform.standardize(np.array(batch_images))
        if self.y is not None:
            batch_y = self.y.loc[batch_indexes].values
            return batch_images, batch_y
        else:
            return batch_images


In [12]:
train_dataset = ImagesDataset_train(x_train, y_train)
eval_dataset = ImagesDataset(x_eval, y_eval)

# efficientnetb4_fine_tuning_마지막 32개 층_batch normalization층 제외하고  동결 해제함 + 층 수정

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications import ResNet50
from efficientnet.tfkeras import EfficientNetB4, preprocess_input

num_classes = 8
# Define the input tensor
input_tensor = Input(shape=(380, 380, 3)) # EfficientNetB4에는 일반적으로 380x380 크기의 입력 이미지가 필요함

# Create the base model
base_model = EfficientNetB4(include_top=False, weights='imagenet', pooling='max') 


71892840/71892840 [==============================] - 1s 0us/step


In [14]:
# Freeze the convolutional base layers
base_model.trainable = True
for layer in base_model.layers[:-32]: # 마지막 7 블록 + Top 블록 > 가중치 동결 해제 (fine_tuning)
  layer.trainable = False


## batch normalization 층은 동결!!!

In [15]:
for layer in base_model.layers:
    if 'BatchNormalization' in layer.__class__.__name__:
        layer.trainable = False

In [16]:
# Pass the input tensor through the base model
base_output = base_model(input_tensor)

# x = layers.GlobalAveragePooling2D()(base_output)
# dense_layer = layers.Dense(128)(x)

# Flatten the output from the base model
flatten_layer = Flatten()(base_output)

# Add a dense layer with 256 units and ReLU activation
dense_layer = Dense(512)(flatten_layer)
dense_layer = Dense(256)(dense_layer)

# Add a dropout layer with dropout rate of 0.5
dropout_layer = Dropout(0.5)(dense_layer)

# Add the final output layer with softmax activation
output_layer = Dense(num_classes, activation='softmax')(dropout_layer)

# Create the model
model = Model(inputs=input_tensor, outputs=output_layer)

# Compile the model
optimizer = Adam(learning_rate=0.001) # 다른 옵션 > optimizer = RMSprop(learning_rate=0.001) / SGD(learning_rate=0.0001, momentum=0.9) / optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [17]:
from tensorflow import keras
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=4,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True,
    )
]\


num_epochs = 100

history = model.fit(train_dataset, epochs=num_epochs, 
                    callbacks=callbacks_list,
                    validation_data=eval_dataset)
  

Epoch 1/100
386/386 [==============================] - 1997s 5s/step - loss: 62.7205 - accuracy: 0.1465 - val_loss: 2.1179 - val_accuracy: 0.1187
Epoch 2/100
386/386 [==============================] - 122s 314ms/step - loss: 2.1754 - accuracy: 0.1465 - val_loss: 2.0844 - val_accuracy: 0.1499
Epoch 3/100
386/386 [==============================] - 120s 311ms/step - loss: 2.1494 - accuracy: 0.1462 - val_loss: 2.0878 - val_accuracy: 0.1450
Epoch 4/100
386/386 [==============================] - 121s 313ms/step - loss: 2.1437 - accuracy: 0.1376 - val_loss: 2.2201 - val_accuracy: 0.1450
Epoch 5/100
386/386 [==============================] - 122s 317ms/step - loss: 2.1806 - accuracy: 0.1365 - val_loss: 2.0599 - val_accuracy: 0.1450
Epoch 6/100
386/386 [==============================] - 135s 350ms/step - loss: 2.1352 - accuracy: 0.1397 - val_loss: 2.0632 - val_accuracy: 0.1343
Epoch 7/100
386/386 [==============================] - 121s 313ms/step - loss: 2.1256 - accuracy: 0.1467 - val_loss: 2.

In [18]:
# 콜백에 저장된 best 모델을 불러오는 것
best_model = keras.models.load_model("checkpoint_path.keras")

mpath = "/content/drive/MyDrive/CAU_BURGER/model/"

In [19]:
# 모델을 지정된 경로에 저장하는 것
best_model.save(mpath+'EfficientNetB4_haram_3.h5')

In [20]:
#  저장된 모델을 model_best 객체에 불러오는 것!!!!
from tensorflow import keras
model_best = keras.models.load_model(mpath+"EfficientNetB4_haram_3.h5")

model_best.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 380, 380, 3)]     0         
                                                                 
 efficientnet-b4 (Functional  (None, 1792)             17673816  
 )                                                               
                                                                 
 flatten (Flatten)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 512)               918016    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                             

In [ ]:
# 여기서부터 제출 양식 만들기
x_test = test_features.filepath.to_frame()
test_dataset = ImagesDataset(x_test)

test_preds = model_best.predict(test_dataset)

 40/279 [===>..........................] - ETA: 15:10

In [ ]:
submission_df = pd.DataFrame(test_preds, index=x_test.index[:test_preds.shape[0]], columns=train_labels.columns)

submission_df

In [ ]:
submission_format = pd.read_csv(dpath+"submission_format.csv", index_col="id")

assert all(submission_df.index == submission_format.index)
assert all(submission_df.columns == submission_format.columns) 

In [ ]:
submission_df.to_csv("/content/drive/MyDrive/CAU_BURGER/submission/EfficientNetB4_haram_3.csv")